In [ ]:
from scipy.spatial import distance as dist
import numpy as np
from collections import OrderedDict


class PersonTracker():
    


    def __init__(self, maxDisappeared=200):
        self.person_ID = 0
        self.objects = OrderedDict()
        self.disappeared = OrderedDict()
        self.maxDisappeared = maxDisappeared

    def register(self, centroid):
        self.objects[self.person_ID] = centroid
        self.disappeared[self.person_ID] = 0
        self.person_ID += 1

    def deregister(self, objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]

    def update(self, rects, tracked_persons):
#         Обработка если не ббоксов
#         if len(rects) == 0:
#             for objectID in self.objects.keys():
#                 self.disappeared[objectID] += 1

#                 if self.disappeared[objectID] > self.maxDisappeared:
#                     self.deregister(objectID)

#             return self.objects

        inputCentroids = np.zeros((len(rects), 2), dtype="int")
        
        tracked_persons = []
        for (i, (startX, startY, endX, endY)) in enumerate(rects):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
            tracked_persons.append(Person.current_face_centroid = (cX, cY))
            
        if len(self.objects) == 0:
            for i in range(0, len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs = list(self.objects.keys())
            objectCentroids = list(self.objects.values())

            D = dist.cdist(np.array(objectCentroids), inputCentroids)
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]

            usedRows = set()
            usedCols = set()

            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue

                objectID = objectIDs[row]
                self.objects[objectID] = inputCentroids[col]
                self.disappeared[objectID] = 0

                usedRows.add(row)
                usedCols.add(col)


            unusedRows = set(range(0, D.shape[0])).difference(usedRows)
            unusedCols = set(range(0, D.shape[1])).difference(usedCols)

            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1

                    if self.disappeared[objectID] > self.maxDisappeared:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])

        return self.objects




In [164]:
import numpy as np

class Person:
    def __init__(self, person_ID):
        self.person_ID = person_ID  # Person face in frame right now
        self.present = False  # Person face in frame right now
        self.current_face_centroid = (None, None)  # Centroid coords in frame
        self.current_face_bbox = (None, None)  # Centroid coords in frame
        self.present_time = None  # time.time()
        self.last_face_centroid = (None, None)  # Last centroid coords when person was detected
        self.gone_frames = 0
        # Emotions
        self.looking = 0
        self.positive = 0
        self.negative = 0
        self.neutral = 0
        self.current_sentiment = ''
        # Head pose
        self.head_pose = (None, None, None)  # yaw pitch roll
        # Gaze
        self.gaze = (None, None)  # yaw pitch roll


rects = [[1225, 622, 1315, 712], [929, 516, 982, 576]]
# List of centroids coords (cX, cY) 
inputCentroids = [(int((startX + endX) / 2), int((startY + endY) / 2.0)) for startX, startY, endX, endY in rects]

tracked_persons = []
if len(tracked_persons) == 0:
    tracked_persons = [Person(num) for num in range(len(rects))]
    for num in range(len(rects)):
        tracked_persons[num].current_face_centroid = inputCentroids[num]
        tracked_persons[num].last_face_centroid = inputCentroids[num]
        tracked_persons[num].current_face_bbox = rects[num]
        tracked_persons[num].gone_frames = 0


In [165]:
num = 0
print(tracked_persons[num].person_ID)
print(tracked_persons[num].current_face_bbox)
print(tracked_persons[num].current_face_centroid)
print(tracked_persons[num].last_face_centroid)
print(tracked_persons[num].gone_frames)

0
[1225, 622, 1315, 712]
(1270, 667)
(1270, 667)
0


### New frame

In [166]:
from scipy.spatial import distance as dist

In [167]:
rects = [[1236, 628, 1321, 711], [521, 519, 571, 574], [930, 515, 983, 577]]
inputCentroids = [(int((startX + endX) / 2), int((startY + endY) / 2.0)) for startX, startY, endX, endY in rects]

maxDisappeared = 400

if len(tracked_persons) != 0:
    objectIDs = [person.person_ID for person in tracked_persons]
    objectCentroids = [person.last_face_centroid for person in tracked_persons]

    D = dist.cdist(np.array(objectCentroids), inputCentroids)
    rows = D.min(axis=1).argsort()   # Индексы строчек по возрастанию с наименьшими расстояниями
    cols = D.argmin(axis=1)[rows]   # Индексы колонок с наименьшими значениями по возрастанию 

    usedRows = set()
    usedCols = set()

    for (row, col) in zip(rows, cols):
        if row in usedRows or col in usedCols:
            continue
        
        # Check if the distance is too big to consider this as the same object
        if D[1, 2] > 100:
            continue

        objectID = objectIDs[row]
        tracked_persons[objectID].current_face_centroid = inputCentroids[col]
        tracked_persons[objectID].last_face_centroid = inputCentroids[col]
        tracked_persons[objectID].gone_frames = 0

        usedRows.add(row)
        usedCols.add(col)

    unusedRows = set(range(0, D.shape[0])).difference(usedRows)
    unusedCols = set(range(0, D.shape[1])).difference(usedCols)

    
    # Если новых МЕНЬШЕ чем старых УДАЛИТЬ объект
    if D.shape[0] >= D.shape[1]:
        for row in unusedRows:
            objectID = objectIDs[row]
            tracked_persons[objectID].gone_frames += 1

            if tracked_persons[objectID].gone_frames > maxDisappeared:
                del tracked_persons[objectID]
              
            
    # Если новых БОЛЬШЕ чем старых ДОБАВИТЬ объект
    else:
        for col in unusedCols:
            new_person_list_id = len(tracked_persons)
            tracked_persons.append(Person(new_person_list_id))
            tracked_persons[new_person_list_id].current_face_bbox = rects[col]
            tracked_persons[new_person_list_id].current_face_centroid = inputCentroids[col]
            tracked_persons[new_person_list_id].last_face_centroid = inputCentroids[col]
            tracked_persons[new_person_list_id].gone_frames = 0

            

In [179]:
D.min(axis=0)

array([  8.24621125, 409.        ,   1.        ])

In [186]:
for (row, col) in zip(rows, cols):
    print(row, col)

1 2
0 0


In [162]:
D.min(axis=1)

array([0., 0., 0.])

In [163]:
min(D[:,col])

0.0

In [161]:
D.argmin(axis=1)[rows]

array([0, 2, 1])

In [147]:
# Обработка исключения, если расстояние до ближайшего незанятого лица больше 100, 
# то не считать это тем же лицом
if min(D[:,col]) > 100:
    contunue 

409.0

In [141]:
D

array([[  8.24621125, 734.04155196, 336.50705788],
       [345.62696654, 409.        ,   1.        ]])

In [138]:
num = 2
print(tracked_persons[num].person_ID)
print(tracked_persons[num].current_face_bbox)
print(tracked_persons[num].current_face_centroid)
print(tracked_persons[num].last_face_centroid)
print(tracked_persons[num].gone_frames)

2
[521, 519, 571, 574]
(546, 546)
(546, 546)
0


In [140]:
tracked_persons